# build an agent 构建代理

In [ ]:
# 导入相关功能模块
from langchain.chat_models import init_chat_model  # 初始化聊天模型
from langchain_tavily import TavilySearch  # Tavily 搜索工具
from langgraph.checkpoint.memory import MemorySaver  # 内存保存器，用于持久化对话状态
from langgraph.prebuilt import create_react_agent  # 创建 ReAct 代理

# 创建代理
memory = MemorySaver()  # 创建内存保存器实例
# model = init_chat_model("anthropic:claude-3-5-sonnet-latest")  # 可选择 Claude 模型
model = init_chat_model("gpt-4o-mini", model_provider="openai")  # 初始化 OpenAI GPT-4o-mini 模型
search = TavilySearch(max_results=2)  # 创建 Tavily 搜索工具，限制最多返回 2 个结果
tools = [search]  # 将搜索工具放入工具列表
agent_executor = create_react_agent(model, tools, checkpointer=memory)  # 创建 ReAct 代理执行器

In [ ]:
# 使用代理
config = {"configurable": {"thread_id": "abc123"}}  # 配置线程 ID，用于区分不同的对话会话

input_message = {
    "role": "user",  # 消息角色为用户
    "content": "Hi, I'm Bob and I live in SF.",  # 用户介绍自己
}
# 流式调用代理，获取响应
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()  # 打印最新的消息

In [ ]:
# 测试代理的记忆能力和工具使用
input_message = {
    "role": "user",
    "content": "What's the weather where I live?",  # 询问天气，代理需要记住用户住在 SF
}

# 代理会自动调用搜索工具查询旧金山的天气
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()  # 显示代理的推理过程和最终回答

### 定义工具

在构建代理之前，我们需要定义代理可以使用的工具。
这里我们使用 Tavily 搜索工具来获取实时信息。

In [ ]:
from langchain_tavily import TavilySearch

# 创建 Tavily 搜索工具实例
search = TavilySearch(max_results=2)  # 限制搜索结果数量为 2
# 测试搜索工具的功能
search_results = search.invoke("What is the weather in SF")
print(search_results)
# 如果需要，我们可以创建其他工具
# 将所有需要的工具放入列表中，供代理使用
tools = [search]  # 工具列表

In [ ]:
from langchain.chat_models import init_chat_model

# 初始化聊天模型（这里使用了一个示例模型名）
model = init_chat_model("gpt-4.1", model_provider="openai")

In [ ]:
# 测试基础模型功能
query = "Hi!"
response = model.invoke([{"role": "user", "content": query}])
response.text()  # 获取模型的文本响应

In [ ]:
# 将工具绑定到模型上
model_with_tools = model.bind_tools(tools)

# 测试绑定工具后的模型行为
query = "Hi!"
response = model_with_tools.invoke([{"role": "user", "content": query}])

# 查看模型的响应内容和工具调用情况
print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

In [ ]:
# 测试需要使用工具的查询
query = "Search for the weather in ShenZhen"
response = model_with_tools.invoke([{"role": "user", "content": query}])

# 这次模型会识别需要调用搜索工具
print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

In [ ]:
from langgraph.prebuilt import create_react_agent

# 创建 ReAct 代理（无内存版本）
agent_executor = create_react_agent(model, tools)

# 测试代理的基本对话功能
input_message = {"role": "user", "content": "Hi!"}
response = agent_executor.invoke({"messages": [input_message]})

# 打印所有消息（包括用户输入和代理响应）
for message in response["messages"]:
    message.pretty_print()

# 测试代理调用工具的能力
input_message = {"role": "user", "content": "Search for the weather in shenzhen"}
response = agent_executor.invoke({"messages": [input_message]})

# 显示完整的对话流程：用户输入 -> 工具调用 -> 工具结果 -> 代理回答
for message in response["messages"]:
    message.pretty_print()

In [ ]:
# 流式消息输出（逐步显示代理的推理过程）
for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"):
    step["messages"][-1].pretty_print()  # 显示每一步的最新消息

In [ ]:
# 流式输出代理的文本响应（实时显示生成的文本）
for step, metadata in agent_executor.stream(
    {"messages": [input_message]}, stream_mode="messages"
):
    # 只显示代理节点生成的文本内容
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")  # 用 | 分隔每个文本块

In [ ]:
# 添加内存功能到代理
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()  # 创建内存保存器

# 创建带有内存功能的代理
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# 配置会话 ID，用于区分不同的对话
config = {"configurable": {"thread_id": "abc123"}}

# 用户自我介绍
input_message = {"role": "user", "content": "Hi, I'm Bob!"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

In [ ]:
# 测试代理的记忆能力
input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()  # 代理应该记住用户的名字是 Bob

In [ ]:
# 使用不同的会话 ID 测试内存隔离
config = {"configurable": {"thread_id": "xyz123"}}

# 在新的会话中询问相同的问题
input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()  # 代理不应该知道用户的名字，因为这是新的会话